In [99]:
# packages

import pandas as pd
import scipy.stats as stats
import numpy as np

In [122]:
# reading data from "National Immunization Survey-Child - Centers for Disease Control and Prevention" (see pdf)

df = pd.read_csv('NISPUF17.csv', index_col = 0)
df.head(5)

,SEQNUMC,SEQNUMHH,PDAT,PROVWT_D,RDDWT_D,STRATUM,YEAR,AGECPOXR,HAD_CPOX,AGEGRP,...,XVRCTY2,XVRCTY3,XVRCTY4,XVRCTY5,XVRCTY6,XVRCTY7,XVRCTY8,XVRCTY9,INS_STAT2_I,INS_BREAK_I
1,128521,12852,2,NaN,235.916956,1031,2017,NaN,2,1,...,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10741,1074,2,NaN,957.353840,1068,2017,NaN,2,1,...,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,220011,22001,2,NaN,189.611299,1050,2017,NaN,2,3,...,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,86131,8613,1,675.430817,333.447418,1040,2017,NaN,2,1,...,,,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0
5,227141,22714,1,482.617748,278.768063,1008,2017,NaN,2,1,...,,,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0


In [134]:
# Write a function called proportion_of_education which returns 
# the proportion of children in the dataset who had a mother with the education levels equal to less than high school (<12), 
# high school (12), more than high school but not a college graduate (>12) and college degree.

# This function should return a dictionary in the form of (use the correct numbers, do not round numbers):

#    {"less than high school":0.2,
#    "high school":0.4,
#    "more than high school but not college":0.2,
#    "college":0.2}.


# Variable we need (Page 42 pdf):  EDUC1 – education of the mother

proportions = df['EDUC1'].value_counts() / len(df)
proportions.head()

4    0.479747
3    0.245881
2    0.172352
1    0.102020
Name: EDUC1, dtype: float64

In [133]:
proportions = df['EDUC1'].value_counts() / len(df)
proportion_of_education = {'less than high school': proportions[1], 'high school': proportions[2],
                              'more than high school but not college': proportions[3], 'college': proportions[4] }
proportion_of_education

{'less than high school': 0.10202002459160373,
 'high school': 0.172352011241876,
 'more than high school but not college': 0.24588090637625154,
 'college': 0.47974705779026877}

In [135]:
# Let's explore the relationship between being fed breastmilk as a child and getting a 
# seasonal influenza vaccine from a healthcare provider. 
# Return a tuple of the average number of influenza vaccines for those children we know 
# received breastmilk as a child and those who know did not.

# This function should return a tuple in the form (use the correct numbers):

#(2.5, 0.1)

# Variable1: CBF_01 – child ever fed breast milk (page 43 pdf)
# Variable2: P_NUMFLU - total number of seasonal influenza dose (page 43 pdf)
df = pd.read_csv('NISPUF17.csv', index_col = 0)

breast_mask = df['CBF_01'] == 1
breast_kids = df[['CBF_01','P_NUMFLU']].where(breast_mask).dropna()
proportions = breast_kids['P_NUMFLU'].sum() / len(breast_kids)

no_breast_mask = df['CBF_01'] == 2
no_breast_kids = df[['CBF_01','P_NUMFLU']].where(no_breast_mask).dropna()
no_proportions = no_breast_kids['P_NUMFLU'].sum() / len(no_breast_kids)

prop = (proportions, no_proportions)
prop

(1.8799187420058687, 1.5963945918878317)

In [137]:
# It would be interesting to see if there is any evidence of a link between vaccine effectiveness and sex of the child. 
# Calculate the ratio of the number of children who contracted chickenpox but were vaccinated against it 
# (at least one varicella dose) versus those who were vaccinated but did not contract chicken pox. Return results by sex.

# This function should return a dictionary in the form of (use the correct numbers):

#    {"male":0.2,
#    "female":0.4}

# Note: To aid in verification, the chickenpox_by_sex()['female'] value the autograder is looking for starts with 
# the digits 0.0077.

# Variable1: HAD_CPOX – did child ever have chicken pox (page 43 pdf)
# Variable2: P_NUMVRC – total number of varicella doses
# Variable3: SEX - sex of child 
df = pd.read_csv('NISPUF17.csv', index_col = 0)
pox = df[['SEX','P_NUMVRC','HAD_CPOX']].dropna()
pox

,SEX,P_NUMVRC,HAD_CPOX
4,2,1.0,2
5,2,0.0,2
7,2,1.0,2
8,2,0.0,2
9,1,0.0,1
...,...,...,...
28447,2,1.0,2
28448,1,1.0,2
28450,1,1.0,2
28453,2,1.0,2


In [155]:
pox_mask = (pox['P_NUMVRC'] > 0) & (pox['HAD_CPOX'] == 1) 
had_pox_and_vaccinated = pox.where(pox_mask).dropna()
proportions1 = had_pox_and_vaccinated['SEX'].value_counts()

no_pox_mask = (pox['P_NUMVRC'] > 0) & (pox['HAD_CPOX'] == 2) 
no_pox_and_vaccinated = pox.where(no_pox_mask).dropna()
proportions2 = no_pox_and_vaccinated['SEX'].value_counts()

prop = proportions1/proportions2

In [158]:
chickenpox_by_sex =  {"male": prop[1] ,"female": prop[2]}
chickenpox_by_sex

{'male': 0.009675583380762664, 'female': 0.0077918259335489565}

In [159]:
# A correlation is a statistical relationship between two variables. If we wanted to know if vaccines work, 
# we might look at the correlation between the use of the vaccine and whether it results in prevention of the infection 
# or disease. In this question, you are to see if there is a correlation between having had the chicken pox 
# and the number of chickenpox vaccine doses given (varicella).

In [160]:
#example
df=pd.DataFrame({"had_chickenpox_column":np.random.randint(1,3,size=(100)),
                   "num_chickenpox_vaccine_column":np.random.randint(0,6,size=(100))})

# here is some stub code to actually run the correlation
corr, pval=stats.pearsonr(df["had_chickenpox_column"],df["num_chickenpox_vaccine_column"])
    
# just return the correlation
print('correlatiosn is', corr, '& pval is',  pval)

correlatiosn is 0.0458872948144587 & pval is 0.6503019858438249


In [165]:
#apllication
df = pd.read_csv('NISPUF17.csv', index_col = 0)

mask_pox = (df['HAD_CPOX'] == 1)|(df['HAD_CPOX'] == 2)

df_new = df[['HAD_CPOX','P_NUMVRC']].where(mask_pox).dropna()
corr, pval=stats.pearsonr(df_new["HAD_CPOX"],df_new["P_NUMVRC"])
# just return the correlation
print('correlatiosn is', corr, '& pval is',  pval)

correlatiosn is 0.07044873460147869 & pval is 2.7780263182892073e-18
